---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [78]:
import pandas as pd
import numpy as np
pd.set_option('display.max_rows', 5000)
pd.set_option('display.max_columns', 5000)
import re
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [79]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [81]:
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ],
    columns=["State", "RegionName"]  )
    The following cleaning needs to be done:
    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''

    df = pd.DataFrame([], columns=['State', 'RegionName'])
    with open('university_towns.txt', 'r') as f:
        state = ""
        for line in f:
            if '[edit]' in line:
                state = line[:line.find('[')].strip()
                continue
            region = line.strip()
            if '(' in region:
                region = region[:(region.find('(') - 1)]
            df = df.append(pd.DataFrame([[state, region]], columns=['State', 'RegionName']), ignore_index=True)
            
            
            
    
    return df.shape
get_list_of_university_towns()

(517, 2)

In [13]:
def get_gdp_df():
    dataset = pd.read_excel("gdplev.xls").iloc[219:,4:-1:2]
    dataset.columns = ["time", "GDP"]
    dataset.reset_index(inplace=True,drop=True)
    for i in range(1,dataset['GDP'].size):
        if dataset.loc[i-1,'GDP']<dataset.loc[i,'GDP']:
            dataset.loc[i,'trend'] = 1
        else:
            dataset.loc[i,'trend'] = 0
    dataset['GDP'] = dataset['GDP'].astype('float64')
    dataset.set_index("time", inplace=True)
    return dataset
get_gdp_df()

,GDP,trend
time,,
2000q1,12359.1,NaN
2000q2,12592.5,1.0
2000q3,12607.7,1.0
2000q4,12679.3,1.0
2001q1,12643.3,0.0
2001q2,12710.3,1.0
2001q3,12670.1,0.0
2001q4,12705.3,1.0
2002q1,12822.3,1.0


In [14]:
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    dataset = get_gdp_df()
    
    #it seems like there is only one recession, so I just make it simple without checking the second recession...
    for i in range(1, len(dataset)-1):
        if ((dataset.iloc[i]['GDP'] < dataset.iloc[i-1]['GDP']) and
            (dataset.iloc[i+1]['GDP'] < dataset.iloc[i]['GDP'])):
            return dataset.index[i]
get_recession_start()

'2008q3'

In [15]:
def get_recession_end():
    dataset = get_gdp_df()
    recession_start = get_recession_start()
    recession_start_index = dataset.index.get_loc(recession_start)
    for i in range(recession_start_index, len(dataset)-1):
        if ((dataset.iloc[i]['GDP'] > dataset.iloc[i-1]['GDP']) and
            (dataset.iloc[i+1]['GDP'] > dataset.iloc[i]['GDP'])):
            return dataset.index[i+1]      
    return recession_start_index
get_recession_end()

'2009q4'

In [17]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    dataset = get_gdp_df()
    recession_start = get_recession_start()
    recession_end = get_recession_end()
    return dataset.loc[recession_start:recession_end]['GDP'].idxmin()
get_recession_bottom()

'2009q2'

In [67]:
def to_quarter(time):
    try:
        time_list = time.split("-")
        if time_list[1] == '01':
            return time_list[0]+'q1'
        elif time_list[1] =='04':
            return time_list[0]+'q2'
        elif time_list[1] =='07':
            return time_list[0]+'q3'
        else:
            return time_list[0]+'q4'
    except:
        print(time_list)
        
    
    

In [74]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    df = pd.read_csv('City_Zhvi_AllHomes.csv').drop(['RegionID', 'Metro', 'CountyName', 'SizeRank'], axis=1)
    df = df.replace({'State': states}).set_index(['State','RegionName']).loc[:,'2000-01':].sort_index()
    count = 0
    while count < 66:
        target = df.columns[0]
        mean_values = np.mean(df.iloc[:,0:3],axis=1)
        df[to_quarter(target)] = mean_values
        df.drop(columns = [df.columns[0],df.columns[1],df.columns[2]], inplace=True)
        count+=1
    target = df.columns[0]
    mean_values = np.mean(df.iloc[:,0:0+2],axis=1)
    df[to_quarter(target)] = mean_values
    df.drop(columns = [df.columns[0],df.columns[1]], inplace=True)
    

    
    return df
df = convert_housing_data_to_quarters()
df

2000q1         2000q2  \
State     RegionName                                              
Alabama   Adamsville                69033.333333   69166.666667   
          Alabaster                122133.333333  123066.666667   
          Albertville               73966.666667   72600.000000   
          Arab                      83766.666667   81566.666667   
          Ardmore                            NaN            NaN   
          Axis                               NaN            NaN   
          Baileyton                          NaN            NaN   
          Bay Minette               81700.000000   78533.333333   
          Bayou La Batre            44066.666667   44500.000000   
          Bessemer                           NaN            NaN   
          Birmingham                54033.333333   54400.000000   
          Boaz                      70866.666667   70266.666667   
          Brent                     92933.333333   94333.333333   
          Brighton                           NaN            NaN   
          Brookwood                 92566.666667   95100.000000   
          Buhl                      90800.000000   94600.000000   
          Calera                   108933.333333  110366.666667   
          Center Point              80966.666667   81233.333333   
          Centreville               95300.000000   96566.666667   
          Chalkville                94100.000000   94433.333333   
          Chancellor                         NaN            NaN   
          Chelsea                  162066.666667  167033.333333   
          Chickasaw                 51200.000000   53666.666667   
          Chunchula                 80266.666667   81766.666667   
          Citronelle                64833.333333   66633.333333   
          Clay                     120900.000000  122266.666667   
          Coden                     62600.000000   64800.000000   
          Coker                    118100.000000  120766.666667   
          Concord                   78600.000000   78700.000000   
          Cottondale               100833.333333  102633.333333   
          Crane Hill                         NaN            NaN   
          Creola                    77066.666667   78233.333333   
          Cullman                            NaN            NaN   
          Daphne                   123166.666667  120233.333333   
          Dauphin Island           177700.000000  176466.666667   
          Deatsville               112733.333333  113500.000000   
          Decatur                            NaN            NaN   
          Duncanville              108100.000000  112033.333333   
          Edgewater                          NaN            NaN   
          Eight Mile                69000.000000   69033.333333   
          Elberta                  134700.000000  147033.333333   
          Elmore                             NaN            NaN   
          Fairfield                 59333.333333   59100.000000   
          Fairhope                 161400.000000  161933.333333   
          Foley                    100233.333333  106100.000000   
          Forestdale                88966.666667   89500.000000   
          Fosters                  107500.000000  110200.000000   
          Fultondale                90066.666667   88366.666667   
          Gardendale               114200.000000  114366.666667   
          Good Hope                          NaN            NaN   
          Grand Bay                 73433.333333   77700.000000   
          Grant                     85066.666667   82333.333333   
          Grayson Valley            88100.000000   89366.666667   
          Graysville                54866.666667   54933.333333   
          Gulf Shores              156500.000000  158900.000000   
          Guntersville              92933.333333   93500.000000   
          Gurley                             NaN            NaN   
          Harpersville              98700.000000   98833.333333   
          Hartselle   

In [72]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    df = pd.read_csv('City_Zhvi_AllHomes.csv')
    df = (df.drop(['RegionID', 'Metro', 'CountyName', 'SizeRank'], axis=1)
             .replace({'State': states})
             .set_index(['State', 'RegionName'])
             .replace(to_replace='NaN', value=np.NaN)
             .convert_objects(convert_numeric=True)
             .sort())
    index = list(df.columns.values).index('2000-01')
    df = df.drop(df.columns[:index], axis=1)
    l = len(df.columns)
    i = 0
    while i < l:
        col_name = df.iloc[:, i].name
        year = int(col_name.split('-')[0])
        month = int(col_name.split('-')[1])
        quarter = get_quarter(year, month)
        if i + 3 < l:
            split = df.iloc[:, i:i + 3]
        else:
            split = df.iloc[:, i:l]
        df[quarter] = split.mean(axis=1)
        i += 3
    df = df.drop(df.columns[:l], axis=1)
    return df
convert_housing_data_to_quarters()

C:\Users\wangh\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: FutureWarning: convert_objects is deprecated.  To re-infer data dtypes for object columns, use DataFrame.infer_objects()
For all other conversions use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  from ipykernel import kernelapp as app


AttributeError: 'DataFrame' object has no attribute 'sort'

In [51]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    
    return "ANSWER"